In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.1.1


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/imnoobud/anaconda3/envs/mlops_lab/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/imnoobud/anaconda3/envs/mlops_lab/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
year = 2021
month = 2

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [13]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [14]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

# Mean predicted

In [16]:
y_pred.mean()

16.191691679979066

# Save results 

In [30]:
df_result = pd.DataFrame(df['ride_id'])
df_result['predicitions'] = y_pred

In [32]:
output_file = './prediction_results.parquet'

In [34]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [35]:
!ls

Dockerfile  model.bin  prediction_results.parquet  starter.ipynb


In [36]:
!ls -lh

total 19M
-rw-rw-r-- 1 imnoobud imnoobud  72 Jun 27 21:46 Dockerfile
-rw-rw-r-- 1 imnoobud imnoobud 18K Jun 27 21:46 model.bin
-rw-rw-r-- 1 imnoobud imnoobud 19M Jun 27 22:06 prediction_results.parquet
-rw-rw-r-- 1 imnoobud imnoobud 17K Jun 27 22:04 starter.ipynb


# Convert notebook 

In [37]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1518 bytes to starter.py


In [38]:
!ls

Dockerfile  model.bin  prediction_results.parquet  starter.ipynb  starter.py


# Deps 

In [43]:
!python --version

Python 3.8.13
